### Q1.Explain the concept of score matching and describe how it is used in score-based (diffusion) generative models.

### Concept of Score Matching

#### Goal

在generative model中，我們想學習資料的分布情況(pdf) \
$p(x) = \frac{1}{Z(\theta)} e^{q(x;\theta)}$ \
但是$Z(\theta)$非常難求解，因此使用其他方法


#### Score function

對$p(x)$取log計算，會得到$\log p(x;\theta)=q(x;\theta) -\log z(\theta)$ \
令$S(x)=\nabla_x\log p(x;\theta) = \nabla_x\log q(x;\theta)$，$S(x)$被稱為Score function. \
能找出在每個點上，機率密度函數上升最快的方向。

#### Explict score matching(ESM)

想訓練模型$S(x;\theta) \approx \nabla_x \log p(x)$ \
因此定義$L_{ESM}(\theta) = \mathbb{E}_{x\sim p(x)}\|S(x; \theta)-\nabla_x\log p(x)\|^2$ \
但我們不知道真實的梯度($\nabla_x\log p(x)$)

#### Implicit Score Matching (ISM)

定義$L_{ISM}(\theta) = \mathbb{E}_{x\sim p(x)}\left[\|S(x;\theta)\|^2 +2\nabla_x\cdot S(x;\theta)\right].$ 

使用這個方法就不需計算出$\nabla_x\log p(x)$

透過計算可以推出： \
$\mathbb{E}_{x\sim p(x)}\left[\|S(x;\theta)-\nabla_x\log p(x)\|^2\right] = \mathbb{E}_{x\sim p(x)}\left[\left(\|S(x;\theta)\|^2+2\nabla_x\cdot S(x;\theta)\right)\right]+\mathbb{E}_{x\sim p(x)}\left[\|\nabla_x\log p(x)\|^2\right]$.



如果訓練出$S(x;\theta)=\nabla_x\log p(x;\theta)$時，$L_{ESM}=0$, $L_{ISM} \leq 0$


#### Denoising score matching (DSM)

在實作中，ISM的計算也沒有那麼方便。
因此後來又提出了一個更簡單的方法：

先對資料$x$加上noise(更好計算)，希望求出noisy score function $S_\sigma(x; \theta)=\nabla_{x}\log p_\sigma(x) \quad$, $p_\sigma(x) = \int_{\mathbb{R}^d}p(x|x_0)p_0(x_0)\,dx_0$

因此定義$L_{DSM}(\theta) = \mathbb{E}_{x_0\sim p_0(x_0)}\mathbb{E}_{x|x_0\sim p(x|x_0)}\left[\|S_\sigma(x;\theta)-\nabla_{x}\log p(x|x_0)\|^2\right].$

#### Why Use DSM

Adding Gaussian noise with isotropic variance. \
$x = x_0 + \epsilon_\sigma, \quad \epsilon_\sigma\sim \mathcal{N}(0, \sigma^2 I),\\
\quad = x_0 + \sigma\epsilon, \quad \epsilon\sim \mathcal{N}(0, I),$

會得到$\nabla_x\log p(x|x_0)= -\frac{(x-x_0)}{\sigma^2} $

帶入公式得到$L_{DSM}(\theta) = \mathbb{E}_{x_0\sim p_0(x_0)}\mathbb{E}_{x|x_0\sim p(x|x_0)}\left[\|S_\sigma(x;\theta)+\frac{(x-x_0)}{\sigma^2}||\right].$

更容易做計算。

#### Sliced Score Matching（SSM）

在高維度中，$\nabla_xS(x;\theta)$是一個巨大的矩陣
想計算Trace,必須計算非常多偏導數

因此使用了Hutchinson’s trace estimator \
當$\mathbb{E}[vv^T] = I$，則$\text{tr}(A) = \mathbb{E}_[v^TAv].$


使用這個方法將ISM改寫得到
$L_{SSM}(\theta) = \mathbb{E}_{x\sim p(x)}\|S(x;\theta)\|^2 +\mathbb{E}_{x\sim p(x)} \mathbb{E}_{v\sim p(v)} \left[2v^T\nabla_x (v^TS(x;\theta))\right].$

---

#### Conclusion

在 generative models 中，我們想學習資料的機率密度函數 (pdf)，  
但直接估計 $p(x;\theta)$ 很困難。  
因此改學它的對數梯度：
$$
S(x) = \nabla_x \log p(x;\theta)
$$
稱為 **score function**。  
我們希望訓練模型 $S_\theta(x) \approx \nabla_x \log p(x)$。

---
#### ESM → ISM

最直接的作法是 **Explicit Score Matching (ESM)**，  
但它需要知道真實分布的梯度 $\nabla_x \log p(x)$，這在實際上不可行。  

因此提出 **Implicit Score Matching (ISM)**  
它透過數學轉換，將目標改寫為不依賴真實梯度的形式，模型就能只靠資料樣本來訓練。

---

#### ISM → DSM

ISM 訓練不穩定，且模型只在資料點附近有效。  
為了讓模型能更穩定地訓練，提出了DSM。

DSM 的想法是：  
對資料加上高斯噪音 $\tilde{x} = x + \epsilon, \; \epsilon \sim \mathcal{N}(0, \sigma^2 I)$，  
然後學習 noisy score

---

#### ISM → SSM
為了在高維度中有效估計 $\nabla_xS_\theta(x)$，   
利用 Hutchinson’s trace estimator 近似：
$$
\nabla_x \cdot S_\theta(x)
= \mathbb{E}_{v\sim \mathcal{N}(0,I)} [v^T \nabla_x (v^T S_\theta(x))]
$$
這樣可以大幅減少計算量。

---



### Q2 Unanswered questions

1.在操作DSM時，我們要如何去選擇noise？能夠讓我們方便計算且結果準確。
2.我們可以結合DSM與SSM的觀念，將noise與轉換成vector的觀念結合再一起，做出效果可能更好的方法嗎？